In [2]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 69.9 MB/s eta 0:00:00


In [3]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from gensim.models import Word2Vec
import warnings

# 경고 메시지 무시 (gensim 관련)
warnings.filterwarnings('ignore')

In [8]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [6]:

# --- 2. 학습용 데이터 (Corpus) 준비 ---
# Word2Vec는 '문맥'을 학습해야 하므로, 여러 문장이 필요합니다.
corpus_text = """
King Henry VIII was the ruler of England. Queen Elizabeth I was his daughter.
The man walked down the street. The woman watched him from the window.
France is a country in Europe. Paris is the capital of France.
Germany is also a country in Europe. Berlin is the capital of Germany.
I enjoy eating a fresh apple. An orange is also a tasty fruit.
"""

print("--- 원본 텍스트 ---")
print(corpus_text)

--- 원본 텍스트 ---

King Henry VIII was the ruler of England. Queen Elizabeth I was his daughter.
The man walked down the street. The woman watched him from the window.
France is a country in Europe. Paris is the capital of France.
Germany is also a country in Europe. Berlin is the capital of Germany.
I enjoy eating a fresh apple. An orange is also a tasty fruit.



In [9]:

# --- 3. 텍스트 전처리 ---
# Word2Vec는 '문장 리스트'를 입력으로 받으며,
# 각 문장은 '토큰 리스트'여야 합니다.
# (예: [['king', 'henry', ...], ['queen', 'elizabeth', ...], ...])

sentences = sent_tokenize(corpus_text.lower())
tokenized_sentences = []
for sentence in sentences:
  tokens = word_tokenize(sentence)
  # 알파벳으로만 구성된 단어만 사용
  cleaned_tokens = [word for word in tokens if word.isalpha()]
  if cleaned_tokens: # 빈 리스트가 아닌 경우에만 추가
    tokenized_sentences.append(cleaned_tokens)

print("\n--- 전처리된 문장 (토큰 리스트의 리스트) ---")
print(tokenized_sentences)


--- 전처리된 문장 (토큰 리스트의 리스트) ---
[['king', 'henry', 'viii', 'was', 'the', 'ruler', 'of', 'england'], ['queen', 'elizabeth', 'i', 'was', 'his', 'daughter'], ['the', 'man', 'walked', 'down', 'the', 'street'], ['the', 'woman', 'watched', 'him', 'from', 'the', 'window'], ['france', 'is', 'a', 'country', 'in', 'europe'], ['paris', 'is', 'the', 'capital', 'of', 'france'], ['germany', 'is', 'also', 'a', 'country', 'in', 'europe'], ['berlin', 'is', 'the', 'capital', 'of', 'germany'], ['i', 'enjoy', 'eating', 'a', 'fresh', 'apple'], ['an', 'orange', 'is', 'also', 'a', 'tasty', 'fruit']]


In [10]:

# --- 4. Word2Vec 모델 학습 ---
print("\n--- Word2Vec 모델 학습 시작 ---")

# Word2Vec 모델을 초기화하고 학습시킵니다.
# vector_size=100 : 단어를 100차원의 벡터로 표현
# window=5 : 현재 단어를 예측하기 위해 앞뒤 5개의 단어를 봄
# min_count=1 : 최소 1번 이상 등장한 단어만 학습
# sg=1 : Skip-Gram 모델 사용 (CBOW(0)보다 성능이 좋은 경향이 있음)
model = Word2Vec(
  sentences=tokenized_sentences,
  vector_size=100,
  window=5,
  min_count=1,
  sg=1, # 1: Skip-Gram, 0: CBOW
  workers=4 # 학습 시 사용할 CPU 코어 수
)

# 학습된 어휘 수 확인
vocab = list(model.wv.key_to_index.keys())
print(f"학습 완료. 총 {len(vocab)}개의 단어에 대한 임베딩 생성.")
print("---------------------------------------")



--- Word2Vec 모델 학습 시작 ---
학습 완료. 총 41개의 단어에 대한 임베딩 생성.
---------------------------------------


In [11]:
# --- 5. 학습된 임베딩 활용 (과제 수행) ---

# 5.1. 특정 단어의 임베딩 벡터 확인
try:
  king_vector = model.wv['king']
  print(f"\n'king'의 임베딩 벡터 (처음 5개 차원):")
  print(king_vector[:5])
  print(f"벡터 차원 수: {king_vector.shape[0]}")
except KeyError:
  print("\n'king'이 어휘 사전에 없습니다.")




'king'의 임베딩 벡터 (처음 5개 차원):
[-0.00479407 -0.00431655 -0.0048006  -0.00980397 -0.00077741]
벡터 차원 수: 100


In [12]:

# 5.2. 유사한 단어 찾기 (model.wv.most_similar)
print("\n--- 'king'과 가장 유사한 단어 Top 5 ---")
#
try:
  similar_to_king = model.wv.most_similar('king', topn=5)
  for word, score in similar_to_king:
    print(f"  {word}: {score:.4f}")
except KeyError:
  print("'king'이 어휘 사전에 없습니다.")

print("\n--- 'paris'와 가장 유사한 단어 Top 5 ---")
try:
  similar_to_paris = model.wv.most_similar('paris', topn=5)
  for word, score in similar_to_paris:
    print(f"  {word}: {score:.4f}")
except KeyError:
  print("'paris'이 어휘 사전에 없습니다.")




--- 'king'과 가장 유사한 단어 Top 5 ---
  queen: 0.2598
  woman: 0.2598
  window: 0.2220
  was: 0.1215
  is: 0.1178

--- 'paris'와 가장 유사한 단어 Top 5 ---
  orange: 0.1966
  is: 0.1891
  man: 0.1425
  down: 0.1365
  window: 0.1074


In [13]:

# 5.3. 단어 간 유사도 계산 (model.wv.similarity)
print("\n--- 단어 간 유사도 계산 (Cosine Similarity) ---")
try:
  # 의미적으로 유사한 단어
  sim_king_queen = model.wv.similarity('king', 'queen')
  print(f"  Similarity('king', 'queen'): {sim_king_queen:.4f}")

  # 의미적으로 관련 없는 단어
  sim_king_apple = model.wv.similarity('king', 'apple')
  print(f"  Similarity('king', 'apple'): {sim_king_apple:.4f}")
except KeyError as e:
  print(f"유사도 계산 실패: {e} 단어가 어휘 사전에 없습니다.")




--- 단어 간 유사도 계산 (Cosine Similarity) ---
  Similarity('king', 'queen'): 0.2598
  Similarity('king', 'apple'): 0.0217


In [14]:



# 5.4. (보너스) 단어 유추 (벡터 연산)
# Word2Vec의 가장 유명한 특징: 'king' - 'man' + 'woman' = ?
print("\n--- 단어 유추 (벡터 연산) ---")
print("  'king' - 'man' + 'woman' = ?")
try:
  #
  result = model.wv.most_similar(positive=['king', 'woman'], negative=['man'], topn=1)
  print(f"  => {result[0][0]} (유사도: {result[0][1]:.4f})")
except KeyError:
  print("  (계산에 필요한 단어가 어휘 사전에 없습니다.)")

print("\n  'paris' - 'france' + 'germany' = ?")
try:
  result = model.wv.most_similar(positive=['paris', 'germany'], negative=['france'], topn=1)
  print(f"  => {result[0][0]} (유사도: {result[0][1]:.4f})")
except KeyError:
  print("  (계산에 필요한 단어가 어휘 사전에 없습니다.)")


--- 단어 유추 (벡터 연산) ---
  'king' - 'man' + 'woman' = ?
  => in (유사도: 0.1891)

  'paris' - 'france' + 'germany' = ?
  => also (유사도: 0.2200)
